In [1]:
import numpy as np
import pandas as pd
import os

from prediction_utils.pytorch_utils.metrics import (
    StandardEvaluator
)

In [2]:
grp_label_dict = {1: "Black women", 2: "White women", 3: "Black men", 4: "White men"}

args = {
    "experiment_name": "apr14_thr",
    "cohort_path": "/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv",
    "base_path": "/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts",
    "eval_fold": "test",
}
aggregate_path = os.path.join(
    args["base_path"], "experiments", args["experiment_name"], "performance", "all"
)

preds_path = os.path.join(aggregate_path, "predictions.csv")
preds = pd.read_csv(preds_path)
eval_df = preds.query('phase == "test"')

In [3]:
eval_df.head()

,phase,outputs,pred_probs,labels,person_id,weights,group,treat,ldlc,relative_risk,fold_id,config_id,model_id,model_type
1906,test,-0.737881,0.023171,0,3,1.069048,2,0,170.600,1.0,1,0,0.001,eqodds_thr
1907,test,-1.241986,0.020980,0,21,1.069048,2,0,170.265,1.0,1,0,0.001,eqodds_thr
1908,test,-0.864410,0.013805,0,42,1.069048,2,0,172.901,1.0,1,0,0.001,eqodds_thr
1909,test,-1.563953,0.015269,0,43,1.069048,2,0,101.509,1.0,1,0,0.001,eqodds_thr
1910,test,-1.316993,0.016703,0,51,1.069048,2,0,134.000,1.0,1,0,0.001,eqodds_thr


In [3]:
evaluator = StandardEvaluator(metrics=['auc', 'loss_bce'])
result_df_ci = evaluator.bootstrap_evaluate(
    df=eval_df,
    n_boot=10,
    strata_vars_eval=['phase', 'model_type', 'fold_id', 'config_id', 'group'],
    strata_vars_boot=['phase', 'labels', 'group'],
    strata_var_replicate='fold_id',
    replicate_aggregation_mode=None,
    strata_var_experiment='config_id',
    baseline_experiment_name=0,
    strata_var_group='group',
    weight_var='weights',
    compute_
#     group_overall_name=-1,
)

In [8]:
result_df_ci.tail()

,phase,model_type,config_id,group,metric,CI_quantile_95,comparator,baseline,delta
235,test,eqodds_thr,9,4,auc,mid,0.786779,0.800736,-0.018435
236,test,eqodds_thr,9,4,auc,upper,0.823090,0.846182,0.005220
237,test,eqodds_thr,9,4,loss_bce,lower,0.295515,0.245561,0.029190
238,test,eqodds_thr,9,4,loss_bce,mid,0.302200,0.267592,0.033955
239,test,eqodds_thr,9,4,loss_bce,upper,0.305799,0.274892,0.050850
